In [32]:
# np.newaxis  -> np 행렬의 차원을 확장하는 함수 
#             -> broadcasting(차원이 다른 행렬간 계산)을 가능하게 해줌
    
# np.repaet   -> (value,num,axis)    value는 반복 생성할 값, num은 생성 횟수, axis는 생성된 값의 추가 방향(축) 

# np.squeeze  -> 크기가 1인 axis를 제거하는 함수  ex)  np.squeeze(a) or a.squeeze()

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from keras import models, layers

from scipy import interpolate
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

In [2]:
sample=pd.read_csv('sample_submission.csv')
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
building_df=pd.read_csv('building_info.csv')

In [3]:
# 종속변수 분리

# elec_power=train_df['전력소비량(kWh)']
# train_df=train_df.drop(['전력소비량(kWh)'],axis=1)

original_consumption=train_df['전력소비량(kWh)'].values

In [96]:
# train_df['date']=train_df['일시'].apply(lambda x:int(x[:8]))
# train_df['time']=train_df['일시'].apply(lambda x:int(x[9:]))

# test_df['date']=test_df['일시'].apply(lambda x:int(x[:8]))
# test_df['time']=test_df['일시'].apply(lambda x:int(x[9:]))

In [9]:
train_df['강수량(mm)']=train_df['강수량(mm)'].fillna(0.0)

In [14]:
# train_df['강수량(mm)'].fillna(0.0, inplace=True)
train_df['풍속(m/s)']=train_df['풍속(m/s)'].fillna(train_df['풍속(m/s)'].mean())
train_df['습도(%)']=train_df['습도(%)'].fillna(train_df['습도(%)'].mean())

In [23]:
train_df.isnull().sum()

num_date_time        0
건물번호                 0
일시                   0
기온(C)                0
강수량(mm)              0
풍속(m/s)              0
습도(%)                0
일조(hr)           75182
일사(MJ/m2)        87913
전력소비량(kWh)           0
dtype: int64

In [24]:
building_df

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [25]:
train_df=train_df.drop(['일조(hr)','일사(MJ/m2)'],axis=1)

In [26]:
# (num_date_time, 일시) column 제거하기 위해 month / day / time(시간)으로 전처리

train_df['month']=train_df['일시'].apply(lambda x:float(x[4:6]))
train_df['day']=train_df['일시'].apply(lambda x:float(x[6:8]))
train_df['time']=train_df['일시'].apply(lambda x:float(x[9:11]))

test_df['month']=test_df['일시'].apply(lambda x:float(x[4:6]))
test_df['day']=test_df['일시'].apply(lambda x:float(x[6:8]))
test_df['time']=test_df['일시'].apply(lambda x:float(x[9:11]))

# train_df=train_df.drop(['일시'],axis=1)
# test_df=test_df.drop(['일시'],axis=1)

# datetime 사용하면 쉽게 변환 가능

In [27]:
test_df['습도(%)']=test_df['습도(%)'].astype('float64')

In [28]:
train_df=pd.merge(train_df,building_df,on='건물번호',how='left')
test_df=pd.merge(test_df,building_df,on='건물번호',how='left')

train_df=train_df.drop(['태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)'],axis=1)
test_df=test_df.drop(['태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)'],axis=1)

train_df=train_df.drop(['건물유형'],axis=1)
test_df=test_df.drop(['건물유형'],axis=1)  # merge후 겹치는 건물 유형 컬럼 제거

In [ ]:
# 불쾌지수 -> [1.8x기온–0.55x(1–습도)x(1.8x기온–26)+32] 

# 새로운 피처(불쾌지수) 생성

In [30]:
train_df = train_df[train_df.columns[:3].to_list() + train_df.columns[8:].to_list() + train_df.columns[3:8].to_list()]
test_df = test_df[test_df.columns[:3].to_list() + test_df.columns[7:].to_list() + test_df.columns[3:7].to_list()]

In [31]:
train_df

,num_date_time,건물번호,일시,month,day,time,연면적(m2),냉방면적(m2),기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,6.0,1.0,0.0,110634.00,39570.00,18.6,0.0,0.9,42.0,1085.28
1,1_20220601 01,1,20220601 01,6.0,1.0,1.0,110634.00,39570.00,18.0,0.0,1.1,45.0,1047.36
2,1_20220601 02,1,20220601 02,6.0,1.0,2.0,110634.00,39570.00,17.7,0.0,1.5,45.0,974.88
3,1_20220601 03,1,20220601 03,6.0,1.0,3.0,110634.00,39570.00,16.7,0.0,1.4,48.0,953.76
4,1_20220601 04,1,20220601 04,6.0,1.0,4.0,110634.00,39570.00,18.4,0.0,2.8,43.0,986.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,8.0,24.0,19.0,57497.84,40035.23,23.1,0.0,0.9,86.0,881.04
203996,100_20220824 20,100,20220824 20,8.0,24.0,20.0,57497.84,40035.23,22.4,0.0,1.3,86.0,798.96
203997,100_20220824 21,100,20220824 21,8.0,24.0,21.0,57497.84,40035.23,21.3,0.0,1.0,92.0,825.12
203998,100_20220824 22,100,20220824 22,8.0,24.0,22.0,57497.84,40035.23,21.0,0.0,0.3,94.0,640.08


In [132]:
# 하이퍼파라미터

input_size = 10  # feature의 개수
hidden_size = 64
num_layers = 3
output_size = 1
num_epochs = 5  
window_size = 24  # 예측에 사용될 시간 윈도우 크기  (하루 시간 단위)
batch_size = 64  # 가중치를 한 번 업데이트 할 횟수의 기준 (64개 학습 후 1번 가중치 갱신)
learning_rate = 0.001

## Dataset

In [133]:
class TimeSeriesDataset(Dataset):
   
    def __init__(self, df, window_size):  # 생성자 / 데이터를 전처리 하는 부분, 필요한 변수 선언
        self.df = df
        self.window_size = window_size
        
    def __len__(self):   # 학습 데이터의 개수를 반환하는 메소드
        return len(self.df) - self.window_size
    
    def __getitem__(self,idx):   # idx에 해당하는 데이터를 반환하는 메소드  / tensor를 return해야 하는 코드 부분
        x = torch.tensor(self.df[idx:idx+self.window_size, :], dtype = torch.float)
        if self.df.shape[1] > 1:
            y = torch.tensor(self.df[idx+self.window_size, -1] ,dtype = torch.float)
        else:
            y = None
        return x, y
    
def create_data_loader(df, window_size, batch_size):
    dataset = TimeSeriesDataset(df, window_size)
    data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=False)  # shuffle은 데이터가 학습되는 순서를 섞는 기능
                                                                                # DataLoader는 mini-batch 만들어주는 역할
    return data_loader

In [134]:
scaler=MinMaxScaler()
train_data = scaler.fit_transform(train_df.drop(['num_date_time', '건물번호', '일시'], axis=1).values)
train_loader=create_data_loader(train_data, window_size, batch_size)

# 최소 0, 최대 1 값으로 정규화 하는 역할로, outlier에 민감한 특징이 있음

## model define

In [61]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()  # 부모 클래스의 생성자를 불러주는 기능  super().__init__()
        
        self.hidden_size=hidden_size
        self.num_layers=num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self,x):
        h0=torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0=torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ =self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'current device',{device})

model=LSTM(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

current device {device(type='cpu')}


## 훈련

In [137]:
# hyper_param={
#     'input_size' : [10],  # feature의 개수
#     'hidden_size' : [64,128],
#     'num_layers' : [3], 
#     'output_size' : [1],
#     'num_epochs' : [5,6,7,8],  
#     'window_size' : [24],  # 예측에 사용될 시간 윈도우 크기  (하루 시간 단위)
#     'batch_size' : [32,64],  # 가중치를 한 번 업데이트 할 횟수의 기준 (64개 학습 후 1번 가중치 갱신)
#     'learning_rate' : [0.001,0.0001]
# }

# 최적의 값을 찾기 위해 다양한 네트워크 구조 조합을 구성할 수도 있다.

In [146]:
train_loader

In [64]:
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.unsqueeze(1).to(device)  # unsqueeze는 차원을 생성하는 함수 <-> squeeze
        
        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and Optimize
        optimizer.zero_grad() #  gradient를 0으로 만들어주는 역할 (원하는 학습의 방향 때문) (pytorch는 미분값이 누적되는 특성 때문)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 300 == 0:
            print('Epoch [{}/{}], step[{}/{}], Loss {:.4f}'
                 .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))


Epoch [1/5], step[300/3188], Loss 0.0004
Epoch [1/5], step[600/3188], Loss 0.0004
Epoch [1/5], step[900/3188], Loss 0.0002
Epoch [1/5], step[1200/3188], Loss 0.0015
Epoch [1/5], step[1500/3188], Loss 0.0146
Epoch [1/5], step[1800/3188], Loss 0.0018
Epoch [1/5], step[2100/3188], Loss 0.0000
Epoch [1/5], step[2400/3188], Loss 0.0001
Epoch [1/5], step[2700/3188], Loss 0.0001
Epoch [1/5], step[3000/3188], Loss 0.0022
Epoch [2/5], step[300/3188], Loss 0.0007
Epoch [2/5], step[600/3188], Loss 0.0003
Epoch [2/5], step[900/3188], Loss 0.0002
Epoch [2/5], step[1200/3188], Loss 0.0006
Epoch [2/5], step[1500/3188], Loss 0.0053
Epoch [2/5], step[1800/3188], Loss 0.0005
Epoch [2/5], step[2100/3188], Loss 0.0000
Epoch [2/5], step[2400/3188], Loss 0.0000
Epoch [2/5], step[2700/3188], Loss 0.0001
Epoch [2/5], step[3000/3188], Loss 0.0003
Epoch [3/5], step[300/3188], Loss 0.0002
Epoch [3/5], step[600/3188], Loss 0.0001
Epoch [3/5], step[900/3188], Loss 0.0000
Epoch [3/5], step[1200/3188], Loss 0.0002
E

## Test_data preprocessing

In [90]:
# 학습 데이터에서 마지막 행 가져오기
last_train_data = train_df.drop(['num_date_time', '건물번호', '일시'], axis=1).loc[204000-24:,:]


# 전력소비량 열 생성
final_df = pd.concat((test_df.drop(['num_date_time', '건물번호', '일시'], axis=1),
                      pd.DataFrame(np.zeros(test_df.shape[0]))),axis=1)
final_df = final_df.rename({0:'전력소비량(kWh)'},axis=1)

In [101]:
new_test_df=pd.concat([last_train_data,final_df]).reset_index(drop=True) #index 순서 0부터 시작하기 위해 
new_test_data=scaler.transform(new_test_df)

In [112]:
new_test_data.shape

(16824, 10)

In [113]:
test_dataset=TimeSeriesDataset(new_test_data,window_size)
test_loader=DataLoader(test_dataset, batch_size=1, shuffle=False)

# 보통 batch_size는 8,16,32... 256의 2^n 단위로 쓰임

## Inference

In [114]:
model.eval()  # train시와 inference시 layer가 다르게 동작하는 것을 방지하기 위함 (ex dropout시)

test_preds=[]

with torch.no_grad():  # autograd를 끔으로써 메모리, 연산시간 save
    for i in range(new_test_data.shape[0] - window_size):
        x = torch.Tensor(new_test_data[i:i+window_size, : ]).to(device)
        new_x = model(x.view(1,window_size,-1))  # .view -> 원소의 수를 유지하면서 텐서의 크기 변경 (np의 reshape와 동일 기능)
        
        new_test_data[i+window_size,-1] = new_x
        test_preds.append(new_x.detach().cpu().numpy().item())
        
        
#  detach().cpu()  -> GPU 메모리에 올려져있는 텐서를 CPU 메모리로 복사하는 함수
#  numpy  -> (여기선 CPU에 올려져있는 텐서에만 적용가능한 함수) numpy로 변환하려면 CPU로 복사해야함

In [116]:
last_prediction=scaler.inverse_transform(new_test_data)[24:,-1]

In [118]:
len(last_prediction)

16800

In [121]:
sample['answer']=last_prediction

In [124]:
sample.to_csv('elec_power_5.csv',index=False)

In [ ]:
# class TimeSeriesDataset(df,window_size):
    
#     def __init__(self,df,window_size):
#         self.df=df
#         self.window_size=window_size
        
#     def __len__(self):
#         return len(self.df) - window_size
    
#     def __getitem__(self,idx):
#         x=torch.tensor(self.df[idx:idx+window_size, : ],dtype=torch.float)
#         if self.df.shape[1] > 1 :
#             y=torch.tensor(self.df[idx+window_size, -1],dtype=torch.float)
#         else:
#             y= None
        
#         return x,y
            

In [ ]:
# name_split = df["full_name"].str.split(" ")
# df["first_name"] = name_split.str.get(0)
# df["last_name"] = name_split.str.get(1)

# 데이터 값의 공백을 기준으로 분리하고 따로 가져와서 각각 다른 컬럼에 저장 가능

In [47]:
 # 브로드캐스팅 

# 브로드 캐스팅이란, 텐서의 크기가 맞지 않을 때 자동으로 크기를 조정 하여 연산을 가능하게 하는 역할
# 크기가 다르면 일단 브로드캐스팅 된 후에 연산을 진행


tensor([[ 1.,  4.,  9.],
        [ 4., 10., 18.],
        [ 7., 16., 27.]])